In [7]:
print("hi"*3)

hihihi


In [8]:
%pwd

'c:\\Users\\HP\\Desktop\\Medibot_prod_stage_GenAI\\notebook'

In [9]:
import os

os.chdir("../")

In [10]:
%pwd

'c:\\Users\\HP\\Desktop\\Medibot_prod_stage_GenAI'

In [11]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [13]:
extracted_data = load_pdf(data="Data/")

In [11]:
# extracted_data

In [14]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [17]:
text_chunks = text_split(extracted_data)
print("length of the Text chunks", len(text_chunks))

length of the Text chunks 5860


In [20]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [22]:
embeddings = download_hugging_face_embeddings()

C:\Users\HP\AppData\Local\Temp\ipykernel_17044\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


c:\Users\HP\Desktop\Medibot_prod_stage_GenAI\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
query_result=embeddings.embed_query("What is the capital of France?")

print("lenght of query is", len(query_result))

lenght of query is 384


In [27]:
# query_result

In [ ]:
from dotenv import load_dotenv

load_dotenv()


True

In [30]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [40]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)


index_name = "medicalchatbot"


pc.create_index(
    name=index_name,
    dimension=384,  # Update this to match the embedding dimension
    metric="cosine",  # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "medicalchatbot",
    "metric": "cosine",
    "host": "medicalchatbot-p5wsrdf.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [41]:
import os

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [42]:
#embed each chunk and upsert the embeddings into Pinecone index
from langchain_pinecone import PineconeVectorStore


docsearch= PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name =index_name,
    embedding=embeddings,
)


In [44]:
# load Existing index 

from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into pinecone index

docsearch= PineconeVectorStore.from_existing_index(
    index_name =index_name,
    embedding=embeddings,
)


In [45]:
docsearch

In [ ]:
retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})    

In [50]:
retriever.invoke("what is acne")

[Document(id='54564b2f-963c-4f34-b795-42338d6790d2', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='ff2f7a21-1151-4cf4-b35b-f1b6c4ebad04', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 37.0, 'page_label': '38', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteri

In [54]:
from dotenv import load_dotenv

load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [135]:
from langchain_groq import ChatGroq

llm=ChatGroq(model='llama-3.3-70b-versatile') #mixtral-8x7b-32768
llm.model_name, llm

('llama-3.3-70b-versatile',
 ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000216BEB89240>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000216BEB8AA40>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********')))

creating rag chain


In [150]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.chains import RetrievalQA
# from langchain.chains.combine_documents import create_stuff_documents_chain

In [137]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt=(
    "you are an assistant for question_answering tasks."
    "use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't knw. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)


In [138]:
template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

In [145]:
question_answer_chain= create_stuff_documents_chain(llm,template)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)

In [142]:
response=rag_chain.invoke({"input":"what is correlation"})


print(response["answer"])

I don't know the definition of correlation from the given context. The provided context does not mention correlation. It discusses various medical and psychological terms, but correlation is not defined.
